# Redes Neuronales Recurrentes


### Paso 1: Preparar los datos

En este ejemplo, utilizaremos el conjunto de datos de IMDB disponible en Keras, que ya está preprocesado y contiene 25,000 reseñas de películas etiquetadas como positivas o negativas.





In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Cargar el conjunto de datos IMDB
vocab_size = 10000  # Número de palabras más frecuentes a considerar
max_sequence_length = 200  # Longitud máxima de las secuencias

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Rellenar las secuencias para que todas tengan la misma longitud
x_train = pad_sequences(x_train, maxlen=max_sequence_length)
x_test = pad_sequences(x_test, maxlen=max_sequence_length)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Paso 2: Definir los modelos RNN, LSTM y GRU

Vamos a definir tres modelos diferentes: uno con una RNN simple, otro con LSTM y otro con GRU.  
Para simplificar, los tres modelos tendrán una estructura similar, con una capa recurrente seguida de una capa densa.

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, Embedding

def create_model(model_type='RNN', vocab_size=10000, max_sequence_length=200, embedding_dim=32, units=50):

  model = Sequential()

  # Capa de embedding para representar palabras como vectores densos
  model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))

  # Capa recurrente: RNN, LSTM o GRU
  if model_type == 'RNN':     model.add(SimpleRNN(units))
  elif model_type == 'LSTM':  model.add(LSTM(units))
  elif model_type == 'GRU':   model.add(GRU(units))

  # Capa densa para la salida binaria
  model.add(Dense(1, activation='sigmoid'))

  # Compilar el modelo
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

### Paso 3: Entrenar los modelos

Ahora entrenaremos los tres modelos y evaluaremos su rendimiento en el conjunto de prueba.

#### Explicación del Código

1. **Carga de los datos**: Usamos el conjunto de datos de IMDB, limitando el vocabulario a las 10,000 palabras más frecuentes.  
Las secuencias se rellenan para que todas tengan la misma longitud (`max_sequence_length`).

2. **Definición del modelo**:
   - La primera capa es una capa `Embedding`, que convierte las palabras en vectores densos de dimensión `embedding_dim`.
   - Dependiendo del tipo de red especificado (`RNN`, `LSTM`, `GRU`), se añade una capa recurrente correspondiente.
   - La capa de salida es una neurona con activación sigmoide para la clasificación binaria.

3. **Entrenamiento de los modelos**: Cada modelo se entrena por separado durante 3 épocas, con un conjunto de validación que corresponde al 20% de los datos de entrenamiento.

4. **Evaluación del rendimiento**: Finalmente, evaluamos cada modelo en el conjunto de prueba para comparar la pérdida y la precisión.

In [3]:
# Entrenamiento y evaluación de cada modelo
for model_type in ['RNN', 'LSTM', 'GRU']:

  print(f'\nEntrenando el modelo { model_type }')
  model = create_model(model_type=model_type)
  model.summary()

  # Entrenar el modelo
  model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2, verbose=1)

  # Evaluar el modelo en el conjunto de prueba
  loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
  print(f'\n{ model_type } - Pérdida en prueba: {loss:.4f}, Precisión en prueba: {accuracy:.4f}')


Entrenando el modelo RNN


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 55s 167ms/step - accuracy: 0.6455 - loss: 0.5894 - val_accuracy: 0.8456 - val_loss: 0.3548
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 170ms/step - accuracy: 0.8977 - loss: 0.2616 - val_accuracy: 0.8644 - val_loss: 0.3231
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 172ms/step - accuracy: 0.9268 - loss: 0.1945 - val_accuracy: 0.8718 - val_loss: 0.3198
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.8635 - loss: 0.3389

GRU - Pérdida en prueba: 0.3361, Precisión en prueba: 0.8658




### Interpretación de Resultados
Al evaluar los tres modelos, podrás notar que las RNN simples tienden a tener un peor rendimiento en tareas que involucran dependencias largas (como análisis de sentimientos), mientras que LSTM y GRU son más efectivas para capturar relaciones a largo plazo en las secuencias. Generalmente, LSTM y GRU tienen rendimientos similares, pero LSTM puede manejar mejor las dependencias complejas en algunos casos.

Este ejemplo proporciona una base sólida para explorar cómo diferentes tipos de redes recurrentes afectan el rendimiento en una tarea de procesamiento de lenguaje natural como la clasificación de sentimientos.

Puedes ajustar hiperparámetros como el número de unidades, la longitud de las secuencias o la cantidad de épocas para experimentar con diferentes configuraciones.

## Ajuste de hiperparámetros

In [4]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

def create_model(model_type='RNN', vocab_size=10000, max_sequence_length=200, embedding_dim=32,
  units=50, dropout_rate=0.2, recurrent_dropout_rate=0.2, learning_rate=0.001):

  model = Sequential()
  # Capa de embedding para representar palabras como vectores densos
  model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))

  # Capa recurrente: RNN, LSTM o GRU con dropout y recurrent_dropout
  if model_type == 'RNN':     model.add(SimpleRNN(units, dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate))
  elif model_type == 'LSTM':  model.add(LSTM(units, dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate))
  elif model_type == 'GRU':   model.add(GRU(units, dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate))

  # Capa densa para la salida binaria
  model.add(Dense(1, activation='sigmoid'))

  # Compilar el modelo con un optimizador Adam personalizado
  optimizer = Adam(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

In [5]:
# Hiperparámetros a ajustar
embedding_dim = 64
units = 100
dropout_rate = 0.3
recurrent_dropout_rate = 0.3
learning_rate = 0.0005
batch_size = 128
epochs = 5
max_sequence_length = 300  # Longitud de la secuencia ajustada

# Ajustar los datos de entrada a la nueva longitud de secuencia
x_train = pad_sequences(x_train, maxlen=max_sequence_length)
x_test = pad_sequences(x_test, maxlen=max_sequence_length)

In [ ]:
# Entrenamiento y evaluación de cada modelo con los nuevos hiperparámetros
for model_type in ['RNN', 'LSTM', 'GRU']:

  print(f'\nEntrenando el modelo { model_type } con hiperparámetros ajustados')
  model = create_model(model_type=model_type, vocab_size=vocab_size, max_sequence_length=max_sequence_length, embedding_dim=embedding_dim,
    units=units, dropout_rate=dropout_rate, recurrent_dropout_rate=recurrent_dropout_rate, learning_rate=learning_rate)

  model.summary()

  # Entrenar el modelo
  model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)

  # Evaluar el modelo en el conjunto de prueba
  loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
  print(f'\n{ model_type } - Pérdida en prueba: {loss:.4f}, Precisión en prueba: {accuracy:.4f}')


Entrenando el modelo RNN con hiperparámetros ajustados


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 151s 932ms/step - accuracy: 0.6111 - loss: 0.6561 - val_accuracy: 0.8234 - val_loss: 0.4093
Epoch 2/5
 15/157 ━━━━━━━━━━━━━━━━━━━━ 2:03 869ms/step - accuracy: 0.8334 - loss: 0.4124